In [3]:
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist
import numpy as np

In [4]:
# 58p
batch_size = 100
original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

In [7]:
def sampling(args: tuple):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0, stddev=epsilon_std)
    
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [8]:
# 인코더 만들기
x = Input(shape=(original_dim), name="input") # 인코더 입력
h = Dense(intermediate_dim, activation="relu", name="encoding")(x) # 중간층
z_mean = Dense(latent_dim, name="mean")(h) # 잠재 공간의 평균을 정의합니다.
z_log_var = Dense(latent_dim, name="log-variance")(h) # 잠재 공간의 로그 분산을 정의합니다.

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var]) # 텐서플로 백엔드를 사용할 때는 output_shape이 꼭 필요한 것은 아닙니다.

encoder = Model(x, [z_mean, z_log_var, z], name="encoder") # 케라스 모델로 인코더를 정의합니다.

In [9]:
# 디코더 만들기
input_decoder = Input(shape=(latent_dim,), name="decoder_input") # 디코더 입력
decoder_h = Dense(intermediate_dim, activation='relu', name="decoder_h")(input_decoder) # 잠재 공간을 중간층의 차원으로 변환합니다.
x_decoded = Dense(original_dim, activation='sigmoid', name="flat_decoded")(decoder_h) # 원본 차원으로 변환합니다.
decoder = Model(input_decoder, x_decoded, name="decoder") # 케라스 모델로 디코더를 정의합니다.

In [10]:
# 모델 결합
output_combined = decoder(encoder(x)[2]) # 인코더의 출력을 디코더에 사용합니다. 인코더의 3번쨰 반환 값이 z입니다.
vae = Model(x, output_combined) # 입력과 출력을 연결합니다.
vae.summary() # 모델 구조를 출력합니다.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [11]:
# 이제부터는 익숙한 부분. 오토인코더를 훈련하기 위해 손실 함수를 정의
kl_loss = -0.5 * K.sum(1 + z_log_var - K.exp(z_log_var) - K.square(z_mean), axis=-1)

vae.add_loss(K.mean(kl_loss) / 784.)
vae.compile(optimizer='rmsprop', loss="binary_crossentropy") # 마지막으로 모델을 컴파일합니다.

In [14]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [15]:
vae.fit(x_train, x_train, shuffle=True, epochs=epochs, batch_size=batch_size)

Epoch 1/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2432
Epoch 2/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2163
Epoch 3/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2122
Epoch 4/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2095
Epoch 5/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2075
Epoch 6/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2059
Epoch 7/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2043
Epoch 8/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2029
Epoch 9/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2016
Epoch 10/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2006
Epoch 11/50
600/600 [==============================] - 2s 3ms/step - loss: 0.1997
Epoch 12/50
600/600 [==============================] - 2s 3ms/step - loss: 0.1990
Epoch 13/50
600/600 [====